# ex06-Reproject DEM

Reprojection is all about changing the coordinates in a dataset from one coordinate system to another. While reprojection is less common these days due to more advanced methods of data distribution, sometimes you will work with multiple rasters that may come with the different projections and your will have to reproject the raster so they are in the same coordinate reference system. Another key reson is that many geospatial operations (such as watershed delineation) only support projected raster data (e.g., DEM), instead of the widely-used projection of latitude and longitude (for short EPSG:4326).

Spatial data from different sources and that cover different extents are often in different Coordinate Reference Systems (CRS). Why is there an incredible variety of map projections? This is because each designed to solve a particular problem.  Every map is a compromise favoring one or more projection properties: area, form (or angle), distance, and direction.

When you meet a DEM with a unexpected projection in practice and you have to reproject it, according to my own experience, the best option is to use a GIS software (such as GRASS GIS, QGIS and SAGA GIS, etc), or apply the ***GDAL*** utility of [gdalwarp](https://gdal.org/programs/gdalwarp.html), which is really the most powerful tool (NOT ONE OF) for reprojecitons. 

Certainly, we also can write a python script to reproject a DEM. For example, the OGR Python API provides a full reprojection support using the Open Spatial Reference module (also known as osr). Hoever, the heavy package provides little abstraction for GDAL’s C API. This means that Python programs using them tend to read and run like C programs. Luckily, another python package of [rasterio](https://rasterio.readthedocs.io/en/stable/intro.html) expresses GDAL’s data model using fewer non-idiomatic extension classes and more idiomatic Python types and protocols, while performing as fast as GDAL’s Python bindings.

This notebook will apply ***rasterio*** to reproject a demo DEM from the projection of EPSG:32613 to EPSG:4326. The demo data can be downloaded from Downloading from https://ndownloader.figshare.com/articles/8259098/versions/2. See more information about EPSG from https://spatialreference.org/ref/epsg/.

In [1]:
%matplotlib inline

import numpy as np
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Reproject DEM with rasterio

Rasterio provides a rasterio.warp.calculate_default_transform() function to determine the optimal resolution and transform for the destination raster. Given a source dataset in a known coordinate reference system, this function will return a transform, width, height tuple which is calculated by libgdal, which makes reprojection easier.

In [2]:
infile  = "data/es_dem/pre_DTM.tif"
outfile = "data/es_dem/pre_DTM_EPSG4326.tif"

***Check original projection***

In [3]:
scr_tif = rio.open(infile)
print(scr_tif.meta)
scr_tif.close()

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 4000, 'height': 2000, 'count': 1, 'crs': CRS.from_epsg(32613), 'transform': Affine(1.0, 0.0, 472000.0,
       0.0, -1.0, 4436000.0)}


***Reproject to EPSG:4326***

In [4]:
dst_crs = 'EPSG:4326' 

with rio.open(infile) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rio.open(outfile, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

***Check new projection***

In [5]:
dst_tif = rio.open(outfile)
print(dst_tif.meta)
print(dst_tif.bounds)
dst_tif.close()

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 4179, 'height': 1614, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(1.124234793876272e-05, 0.0, -105.32837712340124,
       0.0, -1.124234793876272e-05, 40.073923431943214)}
BoundingBox(left=-105.32837712340124, bottom=40.05577828237005, right=-105.28139535136515, top=40.073923431943214)


## References

Pearson, K. (1895). Contributions to the Mathematical Theory of Evolution. II. Skew Variation in Homogeneous Material. Philosophical Transactions of the Royal Society A: Mathematical, Physical and Engineering Sciences. 186: 343–414. Bibcode:1895RSPTA.186..343P. doi:10.1098/rsta.1895.0010

Travis E, Oliphant. A guide to NumPy, USA: Trelgol Publishing, (2006).

Stéfan van der Walt, S. Chris Colbert and Gaël Varoquaux. The NumPy Array: A Structure for Efficient Numerical Computation, Computing in Science & Engineering, 13, 22-30 (2011), DOI:10.1109/MCSE.2011.37

Fernando Pérez and Brian E. Granger. IPython: A System for Interactive Scientific Computing, Computing in Science & Engineering, 9, 21-29 (2007), DOI:10.1109/MCSE.2007.53

John D. Hunter. Matplotlib: A 2D Graphics Environment, Computing in Science & Engineering, 9, 90-95 (2007), DOI:10.1109/MCSE.2007.55

https://rasterio.readthedocs.io/en/stable/intro.html

https://gdal.org/programs/gdalwarp.html

https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/reproject-vector-data/

https://medium.com/planet-stories/a-gentle-introduction-to-gdal-part-2-map-projections-gdalwarp-e05173bd710a